In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 53.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a2258025be1a50b3c727bcbbc0b6e345f4a2f56584cd4ed99e68e36318f61adb
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.17
Branch HEAD
Compiled by user yumwang on 2022-10-15T09:47:01Z
Revision fbbcf9434ac070dd4ced4fb9efe32899c6db12a9
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("CienciaDatos2").master("local[*]").getOrCreate()

In [ ]:
#csv_path = '/content/sample_data/california_housing_train.csv'

In [ ]:
#df = (spark.read.option('header', True)
#    .option('inferSchema', True)
#    .csv(csv_path)
#)

In [ ]:
#df.show(10, False)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31  |34.19   |15.0              |5612.0     |1283.0        |1015.0    |472.0     |1.4936       |66900.0           |
|-114.47  |34.4    |19.0              |7650.0     |1901.0        |1129.0    |463.0     |1.82         |80100.0           |
|-114.56  |33.69   |17.0              |720.0      |174.0         |333.0     |117.0     |1.6509       |85700.0           |
|-114.57  |33.64   |14.0              |1501.0     |337.0         |515.0     |226.0     |3.1917       |73400.0           |
|-114.57  |33.57   |20.0              |1454.0     |326.0         |624.0     |262.0     |1.925        |65500.0           |
|-114.58  |33.63   |29.0

In [5]:
spark = SparkSession.builder.appName("Ciencia Datos Pyspark Tutorial").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

In [7]:
df = spark.read.csv('/content/drive/MyDrive/data/ecommerce.csv',header=True,escape="\"")

InvoiceNo: The unique identifier of each customer invoice.

StockCode: The unique identifier of each item in stock.

Description: The item purchased by the customer.

Quantity: The number of each item purchased by a customer in a single invoice.

InvoiceDate: The purchase date.

UnitPrice: Price of one unit of each item.

CustomerID: Unique identifier assigned to each user.

Country: The country from where the purchase was made.


In [8]:
df.show(5,0)

+---------+---------+-----------------------------------+--------+-------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate  |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |1/12/10 08:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |1/12/10 08:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |1/12/10 08:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |1/12/10 08:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |1/12/10 08:26|3.39     |17850     |United Kingdom|
+---------+---------+-----------------------------------+-------

In [9]:
df.count() 

541909

In [10]:
df.select('CustomerID').distinct().count()

4373

In [11]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [12]:
df.groupBy('Country').agg(countDistinct('CustomerID').alias('country_count')).show()

+------------------+-------------+
|           Country|country_count|
+------------------+-------------+
|            Sweden|            8|
|         Singapore|            1|
|           Germany|           95|
|               RSA|            1|
|            France|           87|
|            Greece|            4|
|European Community|            1|
|           Belgium|           25|
|           Finland|           12|
|             Malta|            2|
|       Unspecified|            4|
|             Italy|           15|
|              EIRE|            3|
|         Lithuania|            1|
|            Norway|           10|
|             Spain|           31|
|           Denmark|            9|
|         Hong Kong|            0|
|            Israel|            4|
|           Iceland|            1|
+------------------+-------------+
only showing top 20 rows



In [13]:
df.groupBy('Country').agg(countDistinct('CustomerID').alias('country_count')).orderBy(desc('country_count')).show()

+---------------+-------------+
|        Country|country_count|
+---------------+-------------+
| United Kingdom|         3950|
|        Germany|           95|
|         France|           87|
|          Spain|           31|
|        Belgium|           25|
|    Switzerland|           21|
|       Portugal|           19|
|          Italy|           15|
|        Finland|           12|
|        Austria|           11|
|         Norway|           10|
|Channel Islands|            9|
|      Australia|            9|
|    Netherlands|            9|
|        Denmark|            9|
|         Cyprus|            8|
|          Japan|            8|
|         Sweden|            8|
|         Poland|            6|
|         Greece|            4|
+---------------+-------------+
only showing top 20 rows



In [14]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df = df.withColumn('date',to_timestamp("InvoiceDate", 'yy/MM/dd HH:mm'))
df.select(max("date")).show()

+-------------------+
|          max(date)|
+-------------------+
|2031-10-11 17:19:00|
+-------------------+



In [15]:
df.select(min("date")).show()

+-------------------+
|          min(date)|
+-------------------+
|0001-02-11 08:23:00|
+-------------------+



In [16]:
df.show(5,0)

+---------+---------+-----------------------------------+--------+-------------+---------+----------+--------------+-------------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate  |UnitPrice|CustomerID|Country       |date               |
+---------+---------+-----------------------------------+--------+-------------+---------+----------+--------------+-------------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |1/12/10 08:26|2.55     |17850     |United Kingdom|0001-12-10 08:26:00|
|536365   |71053    |WHITE METAL LANTERN                |6       |1/12/10 08:26|3.39     |17850     |United Kingdom|0001-12-10 08:26:00|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |1/12/10 08:26|2.75     |17850     |United Kingdom|0001-12-10 08:26:00|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |1/12/10 08:26|3.39     |17850     |United Kingdom|0001-12-10 08:26:00|
|536365   |84029E   |RED WOOLLY HOTTIE WH

RFM se usa comúnmente en marketing para evaluar el valor de un cliente en función de su:

Recency: ¿Qué tan recientemente ha realizado una compra cada cliente?

Frecuencia: ¿Con qué frecuencia han comprado algo?

Valor Monetario: ¿Cuánto dinero gastan en promedio al realizar compras?


**Recency**

In [17]:
df = df.withColumn("from_date", lit("12/1/10 08:26"))
df = df.withColumn('from_date',to_timestamp("from_date", 'yy/MM/dd HH:mm'))

df2=df.withColumn('from_date',to_timestamp(col('from_date'))).withColumn('recency',col("date").cast("long") - col('from_date').cast("long"))

In [18]:
df2 = df2.join(df2.groupBy('CustomerID').agg(max('recency').alias('recency')),on='recency',how='leftsemi')

In [19]:
df2.show(5,0)

+------------+---------+---------+-----------------------------------+--------+-------------+---------+----------+--------------+-------------------+-------------------+
|recency     |InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate  |UnitPrice|CustomerID|Country       |date               |from_date          |
+------------+---------+---------+-----------------------------------+--------+-------------+---------+----------+--------------+-------------------+-------------------+
|-63432109980|536384   |82484    |WOOD BLACK BOARD ANT WHITE FINISH  |3       |1/12/10 09:53|6.45     |18074     |United Kingdom|0001-12-10 09:53:00|2012-01-10 08:26:00|
|-63432109980|536384   |84755    |COLOUR GLASS T-LIGHT HOLDER HANGING|48      |1/12/10 09:53|0.65     |18074     |United Kingdom|0001-12-10 09:53:00|2012-01-10 08:26:00|
|-63432109980|536384   |22464    |HANGING METAL HEART LANTERN        |12      |1/12/10 09:53|1.65     |18074     |United Kingdom|0001-12-10 09:53:00|2

In [20]:
df2.printSchema()

root
 |-- recency: long (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- from_date: timestamp (nullable = true)



**Frequency**

In [21]:
df_freq = df2.groupBy('CustomerID').agg(count('InvoiceDate').alias('frequency'))

In [22]:
df_freq.show(5,0)

+----------+---------+
|CustomerID|frequency|
+----------+---------+
|15574     |1        |
|17714     |10       |
|14157     |1        |
|16250     |10       |
|13772     |36       |
+----------+---------+
only showing top 5 rows



In [23]:
df3 = df2.join(df_freq,on='CustomerID',how='inner')

In [24]:
df3.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- recency: long (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- from_date: timestamp (nullable = true)
 |-- frequency: long (nullable = false)



**Valor Monetario**

In [25]:
m_val = df3.withColumn('TotalAmount',col("Quantity") * col("UnitPrice"))

In [26]:
m_val = m_val.groupBy('CustomerID').agg(sum('TotalAmount').alias('monetary_value'))

In [27]:
finaldf = m_val.join(df3,on='CustomerID',how='inner')

In [28]:
finaldf = finaldf.select(['recency','frequency','monetary_value','CustomerID']).distinct()

**Estandardizacion**

In [29]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

assemble=VectorAssembler(inputCols=[
    'recency','frequency','monetary_value'
], outputCol='features')

assembled_data=assemble.transform(finaldf)

scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)

In [30]:
data_scale_output.select('standardized').show(2,truncate=False)

+----------------------------------------------------------------+
|standardized                                                    |
+----------------------------------------------------------------+
|[0.007053018889234201,0.029659745732762945,0.002761038125761422]|
|[0.01578164180007535,0.29659745732762943,0.11265035553106603]   |
+----------------------------------------------------------------+
only showing top 2 rows



**Machine Learning**

In [31]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import numpy as np

cost = np.zeros(10)

evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized',metricName='silhouette', distanceMeasure='squaredEuclidean')

for i in range(2,10):
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    output=KMeans_fit.transform(data_scale_output)
    cost[i] = KMeans_fit.summary.trainingCost

In [ ]:
import pandas as pd
import pylab as pl
df_cost = pd.DataFrame(cost[2:])
df_cost.columns = ["cost"]
new_col = range(2,10)
df_cost.insert(0, 'cluster', new_col)
pl.plot(df_cost.cluster, df_cost.cost)
pl.xlabel('Number of Clusters')
pl.ylabel('Score')
pl.title('Elbow Curve')
pl.show()

In [ ]:
KMeans_algo=KMeans(featuresCol='standardized', k=4)
KMeans_fit=KMeans_algo.fit(data_scale_output)

In [ ]:
preds=KMeans_fit.transform(data_scale_output)

preds.show(5,0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_viz = preds.select('recency','frequency','monetary_value','prediction')
df_viz = df_viz.toPandas()
avg_df = df_viz.groupby(['prediction'], as_index=False).mean()

list1 = ['recency','frequency','monetary_value']

for i in list1:
    sns.barplot(x='prediction',y=str(i),data=avg_df)
    plt.show()

Aquí hay una descripción general de las características mostradas por los clientes en cada grupo:

Grupo 0: los clientes de este segmento muestran poca actualidad, frecuencia y valor monetario. Rara vez compran en la plataforma y son clientes de bajo potencial que probablemente dejarán de hacer negocios con la empresa de comercio electrónico.

Grupo 1: los usuarios de este grupo muestran una alta antigüedad, pero no se les ha visto gastar mucho en la plataforma. Tampoco visitan el sitio con frecuencia. Esto indica que podrían ser clientes más nuevos que acaban de comenzar a hacer negocios con la empresa.

Grupo 2: los clientes de este segmento muestran una frecuencia y actualidad medias y gastan mucho dinero en la plataforma. Esto indica que tienden a comprar artículos de alto valor o hacer compras al por mayor.

Grupo 3: el segmento final comprende a los usuarios que muestran una alta actualidad y realizan compras frecuentes en la plataforma. Sin embargo, no gastan mucho en la plataforma, lo que podría significar que tienden a seleccionar artículos más baratos en cada compra.